In [1]:
from ray import tune

# 1. Define an objective function.
def objective(config):
    score = config["a"] ** 2 + config["b"]
    return {"score": score}


# 2. Define a search space.
search_space = {
    "a": tune.grid_search([0.001, 0.01, 0.1, 1.0]),
    "b": tune.choice([1, 2, 3]),
}

# 3. Start a Tune run and print the best result.
tuner = tune.Tuner(objective, param_space=search_space)
results = tuner.fit()
print(results.get_best_result(metric="score", mode="min").config)

2023-01-11 17:12:03,097	INFO worker.py:1538 -- Started a local Ray instance.


Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,score,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
objective_b1b87_00000,2023-01-11_17-12-13,True,,0644947c8006415ab358d61c72e8ad05,"0_a=0.0010,b=1",LAPTOP-7VUL2UFI,1,127.0.0.1,11416,1,0.000991344,0.000991344,0.000991344,1673453533,0,,1,b1b87_00000,0.00500941
objective_b1b87_00001,2023-01-11_17-12-17,True,,d988cb58d4b1499b9281ca3c2aeb7f62,"1_a=0.0100,b=2",LAPTOP-7VUL2UFI,1,127.0.0.1,12032,2.0001,0.00200033,0.00200033,0.00200033,1673453537,0,,1,b1b87_00001,0.0060997
objective_b1b87_00002,2023-01-11_17-12-18,True,,d03427aed7ef4082b45bdf942c00101c,"2_a=0.1000,b=2",LAPTOP-7VUL2UFI,1,127.0.0.1,2608,2.01,0,0,0,1673453538,0,,1,b1b87_00002,0.00600719
objective_b1b87_00003,2023-01-11_17-12-17,True,,e20dae91df704ccea200bcc59dfb32a8,"3_a=1.0000,b=1",LAPTOP-7VUL2UFI,1,127.0.0.1,20228,2,0.00101018,0.00101018,0.00101018,1673453537,0,,1,b1b87_00003,0.00631642


2023-01-11 17:12:18,230	INFO tune.py:762 -- Total run time: 11.29 seconds (9.00 seconds for the tuning loop).


{'a': 0.001, 'b': 1}


In [1]:
import torch
from ray import tune, air
from ray.air import session
from ray.tune.search.optuna import OptunaSearch

In [2]:
import torch.nn as nn
from torch.optim import SGD

In [3]:
class Model(nn.Module):
    def __init__(self, inp_size, hidden_size, out_size):
        super().__init__()
        self.i2h = nn.Linear(inp_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, out_size)
        self.sigmoid = nn.LogSigmoid()
    
    def forward(self, inp):
        x = self.i2h(inp)
        x = self.sigmoid(x)
        x = self.h2h(x)
        x = self.sigmoid(x)
        x = self.h2o(x)
        return x

In [7]:
import torchvision
import torchvision.datasets as datasets

In [8]:
batch_size_train = 32
batch_size_test = 32

In [9]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [21]:
for data in test_loader:
    #print(len(data))
    print(data[0].shape)
    print(data[1].shape)
    break

torch.Size([32, 1, 28, 28])
torch.Size([32])


In [10]:
def train(model, optimizer, train_loader):
    loss = nn.CrossEntropyLoss()
    for data in train_loader:
        X, y = data[0].reshape(batch_size_train, -1), data[1]
        out = model(X)
        l = loss(out, y) 
        optimizer.zero_grad()
        l.backward()
        optimizer.step()

In [11]:
from sklearn.metrics import accuracy_score

In [12]:
def test(model, test_loader):
    true_cls = []
    pred_cls = []
    for data in train_loader:
        X, y = data[0].reshape(batch_size_train, -1), data[1]
        out = model(X)
        pred = torch.argmax(out, dim=1)
        true_cls.extend(y.reshape(-1).numpy().tolist())
        pred_cls.extend(pred.reshape(-1).numpy().tolist())
    return accuracy_score(true_cls, pred_cls)    

In [26]:
model = Model(inp_size=784, hidden_size=1568, out_size=10)
optimizer = SGD(model.parameters(), lr=0.0001)

In [27]:
train(model, optimizer, train_loader)

In [37]:
test(model, test_loader)

0.0625

In [13]:
def load_data():
    return train_loader, test_loader

In [ ]:
# 1. Wrap your PyTorch model in an objective function.
import torch
from ray import tune, air
from ray.air import session
from ray.tune.search.optuna import OptunaSearch


# 1. Wrap a PyTorch model in an objective function.
def objective(config):
    train_loader, test_loader = load_data()  # Load some data
    model = Model(inp_size=784, hidden_size=1568, out_size=10).to("cpu")  # Create a PyTorch conv net
    optimizer = torch.optim.SGD(  # Tune the optimizer
        model.parameters(), lr=config["lr"], momentum=config["momentum"]
    )

    while True:
        train(model, optimizer, train_loader)  # Train the model
        acc = test(model, test_loader)  # Compute test accuracy
        session.report({"mean_accuracy": acc})  # Report to Tune


# 2. Define a search space and initialize the search algorithm.
search_space = {"lr": tune.loguniform(1e-4, 1e-2), "momentum": tune.uniform(0.1, 0.9)}
algo = OptunaSearch()

# 3. Start a Tune run that maximizes mean accuracy and stops after 5 iterations.
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_accuracy",
        mode="max",
        search_alg=algo,
    ),
    run_config=air.RunConfig(
        stop={"training_iteration": 5},
    ),
    param_space=search_space,
)
results = tuner.fit()
print("Best config is:", results.get_best_result().config)

2023-01-13 17:43:11,625	INFO worker.py:1538 -- Started a local Ray instance.
C:\Users\ForschungszentrumL3S\anaconda3\lib\site-packages\ray\tune\search\optuna\optuna_search.py:673: FutureWarning: LogUniformDistribution has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :class:`~optuna.distributions.FloatDistribution` instead.
  return ot.distributions.LogUniformDistribution(
C:\Users\ForschungszentrumL3S\anaconda3\lib\site-packages\ray\tune\search\optuna\optuna_search.py:682: FutureWarning: UniformDistribution has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :class:`~optuna.distributions.FloatDistribution` instead.
  return ot.distributions.UniformDistribution(
[I 2023-01-13 17:43:15,432] A new study created in memory with name: optuna


C:\Users\ForschungszentrumL3S\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:766: FutureWarning: LogUniformDistribution(high=0.01, low=0.0001) is deprecated and internally converted to FloatDistribution(high=0.01, log=True, low=0.0001, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
C:\Users\ForschungszentrumL3S\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:766: FutureWarning: UniformDistribution(high=0.9, low=0.1) is deprecated and internally converted to FloatDistribution(high=0.9, log=False, low=0.1, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
2023-01-13 17:43:18,680	WARNING worker.py:1851 -- Warning: The actor ImplicitFunc is very large (52 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
